# Lectura de archivos

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [44]:
id = '1br3fMwXX_J0XmiXvOm_wfKWvHSj45T3y'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_values.csv')

train_values = pd.read_csv('train_values.csv', encoding='latin-1', dtype = {'building_id': 'int64', 'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',\
                                                                           'count_floors_pre_eq': 'int64', 'age': 'int64', 'area_percentage': 'int64', \
                                                                           'height_percentage': 'int64', 'land_surface_condition': 'category',\
                                                                           'foundation_type': 'category', 'roof_type': 'category', 'ground_floor_type': 'category',\
                                                                           'other_floor_type': 'category', 'position': 'category', 'plan_configuration': 'category',\
                                                                           'has_superstructure_adobe_mud': 'boolean', 'has_superstructure_mud_mortar_stone': 'boolean', \
                                                                           'has_superstructure_stone_flag': 'boolean', 'has_superstructure_cement_mortar_stone': 'boolean',\
                                                                           'has_superstructure_mud_mortar_brick': 'boolean', 'has_superstructure_cement_mortar_brick': 'boolean',\
                                                                           'has_superstructure_timber': 'boolean', 'has_superstructure_bamboo': 'boolean',\
                                                                           'has_superstructure_rc_non_engineered': 'boolean', 'has_superstructure_rc_engineered': 'boolean',\
                                                                           'has_superstructure_other': 'boolean', 'legal_ownership_status': 'category', 'count_families': 'int64', \
                                                                           'has_secondary_use': 'boolean', 'has_secondary_use_agriculture': 'boolean', 'has_secondary_use_hotel': 'boolean', \
                                                                           'has_secondary_use_rental': 'boolean', 'has_secondary_use_institution': 'boolean', 'has_secondary_use_school': 'boolean',\
                                                                           'has_secondary_use_industry': 'boolean', 'has_secondary_use_health_post': 'boolean', \
                                                                           'has_secondary_use_gov_office': 'boolean', 'has_secondary_use_use_police': 'boolean', 'has_secondary_use_other': 'boolean'}, index_col='building_id')

In [4]:
id = '1RUtolRcQlR3RGULttM4ZoQaK_Ouow4gc'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_labels.csv')
train_labels = pd.read_csv('train_labels.csv', encoding='latin-1', dtype={'building_id': 'int64', 'damage_grade': 'int64'}, index_col='building_id')

In [5]:
print(train_labels.shape)
print(train_values.shape)

(260601, 1)
(260601, 38)


In [45]:
id = '1kt2VFhgpfRS72wtBOBy1KDat9LanfMZU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_values.csv')

test_values = pd.read_csv('test_values.csv', encoding='latin-1', dtype = {'building_id': 'int64', 'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',\
                                                                           'count_floors_pre_eq': 'int64', 'age': 'int64', 'area_percentage': 'int64', \
                                                                           'height_percentage': 'int64', 'land_surface_condition': 'category',\
                                                                           'foundation_type': 'category', 'roof_type': 'category', 'ground_floor_type': 'category',\
                                                                           'other_floor_type': 'category', 'position': 'category', 'plan_configuration': 'category',\
                                                                           'has_superstructure_adobe_mud': 'boolean', 'has_superstructure_mud_mortar_stone': 'boolean', \
                                                                           'has_superstructure_stone_flag': 'boolean', 'has_superstructure_cement_mortar_stone': 'boolean',\
                                                                           'has_superstructure_mud_mortar_brick': 'boolean', 'has_superstructure_cement_mortar_brick': 'boolean',\
                                                                           'has_superstructure_timber': 'boolean', 'has_superstructure_bamboo': 'boolean',\
                                                                           'has_superstructure_rc_non_engineered': 'boolean', 'has_superstructure_rc_engineered': 'boolean',\
                                                                           'has_superstructure_other': 'boolean', 'legal_ownership_status': 'category', 'count_families': 'int64', \
                                                                           'has_secondary_use': 'boolean', 'has_secondary_use_agriculture': 'boolean', 'has_secondary_use_hotel': 'boolean', \
                                                                           'has_secondary_use_rental': 'boolean', 'has_secondary_use_institution': 'boolean', 'has_secondary_use_school': 'boolean',\
                                                                           'has_secondary_use_industry': 'boolean', 'has_secondary_use_health_post': 'boolean', \
                                                                           'has_secondary_use_gov_office': 'boolean', 'has_secondary_use_use_police': 'boolean', 'has_secondary_use_other': 'boolean'})

In [47]:
#train_values.set_index('building_id', inplace=True)
test_values.set_index('building_id', inplace=True)

In [48]:
train_values_original = train_values
test_values_original = test_values

In [49]:
train_values = train_values_original
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,True,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False
28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False
94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False
590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,False,True,False,False,False,False,True,True,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False
201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,True,False,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False


# Feature Engineering

In [399]:
train_values = train_values_original
test_values = test_values_original
train_values.set_index('building_id', inplace=True)
test_values.set_index('building_id', inplace=True)

##Promedio de edad según la zona geográfica:

In [400]:
def avg_age(df_values):
  geo_levels = df_values[['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'age']]
  geo_levels.reset_index(inplace=True)

  age_avg_1 = geo_levels.groupby('geo_level_1_id')['age'].mean()
  age_avg_2 = geo_levels.groupby('geo_level_2_id')['age'].mean()
  age_avg_3 = geo_levels.groupby('geo_level_3_id')['age'].mean()

  age_avg_1.rename('age_avg_geo_level_1_id', inplace=True)
  age_avg_2.rename('age_avg_geo_level_2_id', inplace=True)
  age_avg_3.rename('age_avg_geo_level_3_id', inplace=True)

  geo_levels = geo_levels\
        .merge(age_avg_1, on='geo_level_1_id')\
        .merge(age_avg_2, on='geo_level_2_id')\
        .merge(age_avg_3, on='geo_level_3_id')

  geo_levels.set_index('building_id', inplace=True)

  new_features = ['building_id', 'age_avg_geo_level_1_id', 'age_avg_geo_level_2_id', 'age_avg_geo_level_3_id']

  geo_levels.reset_index(inplace=True)
  df_values.reset_index(inplace=True)

  df_values = df_values.merge(geo_levels[new_features]).set_index('building_id')
  return df_values

In [401]:
train_values = avg_age(train_values)
train_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,count_materials,families_and_secondary_use,inhabited_building,range_age,age_avg_geo_level_1_id,age_avg_geo_level_2_id,age_avg_geo_level_3_id
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,True,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,2.0,False,False,regular,33.021205,22.203704,23.378378


In [402]:
test_values = avg_age(test_values)
test_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_avg_geo_level_1_id,age_avg_geo_level_2_id,age_avg_geo_level_3_id
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,22.799919,24.666667,33.75


## Cantidad de familias en las zonas geográficas

In [403]:
def count_families(df_values):
  count_families = df_values[['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'count_families']]
  count_families.reset_index(inplace=True)

  families_1 = count_families.groupby('geo_level_1_id')['count_families'].sum()
  families_2 = count_families.groupby('geo_level_2_id')['count_families'].sum()
  families_3 = count_families.groupby('geo_level_3_id')['count_families'].sum()

  families_1.rename('count_families_geo_level_1_id', inplace=True)
  families_2.rename('count_families_geo_level_2_id', inplace=True)
  families_3.rename('count_families_geo_level_3_id', inplace=True)

  count_families = count_families\
        .merge(families_1, on='geo_level_1_id')\
        .merge(families_2, on='geo_level_2_id')\
        .merge(families_3, on='geo_level_3_id')

  count_families.set_index('building_id', inplace=True)

  new_features = ['building_id', 'count_families_geo_level_1_id', 'count_families_geo_level_2_id', 'count_families_geo_level_3_id']

  count_families.reset_index(inplace=True)
  df_values.reset_index(inplace=True)

  df_values = df_values.merge(count_families[new_features]).set_index('building_id')
  return df_values

In [404]:
train_values = count_families(train_values)
train_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,count_materials,families_and_secondary_use,inhabited_building,range_age,age_avg_geo_level_1_id,age_avg_geo_level_2_id,age_avg_geo_level_3_id,count_families_geo_level_1_id,count_families_geo_level_2_id,count_families_geo_level_3_id
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,True,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,2.0,False,False,regular,33.021205,22.203704,23.378378,22906,268,36


In [405]:
test_values = count_families(test_values)
test_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_avg_geo_level_1_id,age_avg_geo_level_2_id,age_avg_geo_level_3_id,count_families_geo_level_1_id,count_families_geo_level_2_id,count_families_geo_level_3_id
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,22.799919,24.666667,33.75,7584,46,4


## Cantidad de edificios por zona geográfica

In [406]:
def count_buildings(df_values):
  count_buildings = df_values[['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'age']]
  count_buildings.reset_index(inplace=True)

  buildings_1 = count_buildings.groupby('geo_level_1_id')['age'].count()
  buildings_2 = count_buildings.groupby('geo_level_2_id')['age'].count()
  buildings_3 = count_buildings.groupby('geo_level_3_id')['age'].count()

  buildings_1.rename('count_buildings_geo_level_1_id', inplace=True)
  buildings_2.rename('count_buildings_geo_level_2_id', inplace=True)
  buildings_3.rename('count_buildings_geo_level_3_id', inplace=True)

  count_buildings = count_buildings\
        .merge(buildings_1, on='geo_level_1_id')\
        .merge(buildings_2, on='geo_level_2_id')\
        .merge(buildings_3, on='geo_level_3_id')

  count_buildings.set_index('building_id', inplace=True)

  new_features = ['building_id', 'count_buildings_geo_level_1_id', 'count_buildings_geo_level_2_id', 'count_buildings_geo_level_3_id']

  count_buildings.reset_index(inplace=True)
  df_values.reset_index(inplace=True)

  df_values = df_values.merge(count_buildings[new_features]).set_index('building_id')
  return df_values

In [407]:
train_values = count_buildings(train_values)
train_values[['count_families_geo_level_3_id', 'count_buildings_geo_level_3_id']].head(10)

,count_families_geo_level_3_id,count_buildings_geo_level_3_id
building_id,,
802906,36,37
28830,17,16
94947,159,136
590882,31,31
201944,124,122
333020,33,38
728451,9,9
475515,18,20
441126,36,37


In [408]:
test_values = count_buildings(test_values)
test_values[['count_families_geo_level_3_id', 'count_buildings_geo_level_3_id']].head(10)

,count_families_geo_level_3_id,count_buildings_geo_level_3_id
building_id,,
300051,4,4
99355,3,3
890251,4,4
745817,210,190
421793,7,7
871976,2,2
691228,22,29
896100,6,8
343471,5,5


 ## Tiene algún material = concreto reforzado

In [409]:
def has_engineered(df_values):
  df_values['has_superstructure_engineered'] = df_values['has_superstructure_rc_non_engineered'] | df_values['has_superstructure_rc_engineered']
  return df_values

In [410]:
train_values = has_engineered(train_values)
train_values[train_values['has_superstructure_engineered']]\
        [['has_superstructure_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_rc_non_engineered']]

,has_superstructure_engineered,has_superstructure_rc_engineered,has_superstructure_rc_non_engineered
building_id,,,
440102,True,False,True
288541,True,True,False
708530,True,True,False
697129,True,False,True
635262,True,True,False
...,...,...,...
353176,True,False,True
344478,True,True,False
155803,True,True,False


In [411]:
test_values = has_engineered(test_values)
test_values[test_values['has_superstructure_engineered']]\
        [['has_superstructure_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_rc_non_engineered']]

,has_superstructure_engineered,has_superstructure_rc_engineered,has_superstructure_rc_non_engineered
building_id,,,
81554,True,False,True
322922,True,False,True
304627,True,False,True
13216,True,False,True
332157,True,False,True
...,...,...,...
424485,True,False,True
806715,True,True,False
660885,True,False,True


## Cantidad de materiales de la edificación

In [412]:
def count_materials(df_values):
  materials = df_values.filter(like='has_superstructure')
  df_values['count_materials'] = materials.sum(axis=1)
  return df_values

In [413]:
train_values = count_materials(train_values)
test_values = count_materials(test_values)

## Edificaciones sin familias residentes 

Esto puede indicar que el estado legal de la tierra es medio trucho, o que la edificación es de muy mala calidad, o muy nueva, o con precios inaccesibles

In [414]:
train_values['inhabited_building'] = train_values['count_families'] == 0
test_values['inhabited_building'] = test_values['count_families'] == 0

In [415]:
#chequeo el estado legal de la tierra para estas edificaciones
train_values[train_values['inhabited_building']]['legal_ownership_status'].value_counts()

v    20049
a      464
w      227
r      122
Name: legal_ownership_status, dtype: int64

Es muuuy mayoritario el estado 'v' -> Esto puede decir algo sobre la tierra

In [416]:
#veo damage_grade
train_values_aux = train_values.reset_index()
buildings = train_values_aux[train_values_aux['legal_ownership_status'] == 'v']['building_id'].tolist()
train_labels.loc[buildings, 'damage_grade'].value_counts()

2    143154
3     84515
1     23270
Name: damage_grade, dtype: int64

No dice nada sobre damage_grade => No es tan wow, pero igual me puede estar diciendo algo piola sobre ese edificio

In [417]:
#chequeo usos secundarios
print(train_values[train_values['inhabited_building'] & (train_values.filter(like='has_secondary_use').sum(axis=1) == 0)].shape)
print(train_values[train_values['inhabited_building'] & (train_values.filter(like='has_secondary_use').sum(axis=1) > 0)].shape)

(16083, 52)
(4779, 52)


Por lo general, no tiene uso secundarios -> Es raro

## Tiene usos secundarios y viven familias

Esto puede indicar algún tipo de precariedad

In [418]:
def families_and_secondary_use(df_values):
  secondary_use = df_values.filter(like='has_secondary_use')
  df_values['families_and_secondary_use'] = (secondary_use.sum(axis=1) > 0) & (df_values['count_families'] > 0)
  return df_values

In [419]:
train_values = families_and_secondary_use(train_values)
train_values[train_values['families_and_secondary_use']].filter(like='has_secondary_use').head(10)

,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,
333020,True,True,False,False,False,False,False,False,False,False,False
973909,True,True,False,False,False,False,False,False,False,False,False
440102,True,False,False,True,False,False,False,False,False,False,False
552720,True,True,False,False,False,False,False,False,False,False,False
961851,True,True,False,False,False,False,False,False,False,False,False
612701,True,True,False,False,False,False,False,False,False,False,False
931198,True,False,True,False,False,False,False,False,False,False,False
697129,True,True,False,False,False,False,False,False,False,False,False
824847,True,False,True,False,False,False,False,False,False,False,False


In [420]:
test_values = families_and_secondary_use(test_values)
test_values[test_values['families_and_secondary_use']].filter(like='has_secondary_use').head(10)

,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,
99355,True,True,False,False,False,False,False,False,False,False,False
745817,True,False,False,True,False,False,False,False,False,False,False
535718,True,False,False,True,False,False,False,False,False,False,False
304627,True,False,True,False,False,False,False,False,False,False,False
13216,True,True,False,False,False,False,False,False,False,False,False
359387,True,True,False,False,False,False,False,False,False,False,False
123399,True,False,True,False,False,False,False,False,False,False,False
675284,True,True,False,False,False,False,False,False,False,False,False
13833,True,True,False,False,False,False,False,False,False,False,False


## Generando rango de edades para la edificación = Binning de edades

* 0 a 5 años -> nueva
* De 5 a 15 años -> actual
* De 15 a 30 años -> regular
* De 30 a 60 años -> regular vieja
* De 60 a 100 años -> vieja
* Mayor que 100 años -> histórica 

In [421]:
pd.qcut(train_values['age'],\
                      q=[0, .2, .4, .8, .95, .99, 1]).value_counts()

(5.0, 15.0]       74906
(15.0, 30.0]      74576
(-0.001, 5.0]     59738
(30.0, 60.0]      38882
(60.0, 100.0]     10375
(100.0, 995.0]     2124
Name: age, dtype: int64

In [422]:
train_values['range_age'] = pd.qcut(train_values['age'],\
                      q=[0, .2, .4, .8, .95, .99, 1],\
                      labels=['new', 'actual', 'regular', 'regular_old', 'old', 'historic'])

test_values['range_age'] = pd.qcut(test_values['age'],\
                      q=[0, .2, .4, .8, .95, .99, 1],\
                      labels=['new', 'actual', 'regular', 'regular_old', 'old', 'historic'])

In [423]:
train_values['range_age'].value_counts()

actual         74906
regular        74576
new            59738
regular_old    38882
old            10375
historic        2124
Name: range_age, dtype: int64

In [424]:
test_values['range_age'].value_counts()

actual         24848
regular        24779
new            20032
regular_old    12930
old             3567
historic         712
Name: range_age, dtype: int64

# Chequeos del proceso de Feature Engineering

## Si quiero hacer un one-hot encoding es muy fácil

In [283]:
#hago train_values[['nueva', 'actual', ...]] = a lo de abajo
pd.get_dummies(train_values['range_age'])

,new,actual,regular,regular_old,old,historic
building_id,,,,,,
802906,0,0,1,0,0,0
28830,0,1,0,0,0,0
94947,0,1,0,0,0,0
590882,0,1,0,0,0,0
201944,0,0,1,0,0,0
...,...,...,...,...,...,...
688636,0,0,0,1,0,0
669485,1,0,0,0,0,0
602512,0,0,0,1,0,0


## Veo que esté bien el shape y las columnas entre los 3 sets

train_values(x) == train_labels(x)

train_values(y) == test_values(x)

In [284]:
print(train_values.shape)
print(train_labels.shape)
print(test_values.shape)

(260601, 52)
(260601, 1)
(86868, 52)


In [285]:
train_values.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [286]:
test_values.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

#Feature Selection

###Método 1:

In [381]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

In [382]:
print(train_values.shape)
print(train_labels.shape)
print(test_values.shape)

(260601, 52)
(260601, 1)
(86868, 52)


Tengo que hacer un one-hot de las categóricas si o si

In [383]:
train_values_aux = train_values

In [384]:
train_values[pd.get_dummies(train_values.select_dtypes('category')).columns] = pd.get_dummies(train_values.select_dtypes('category')) 

In [340]:
#train_values[pd.get_dummies(train_values.select_dtypes('boolean')).columns] = pd.get_dummies(train_values.select_dtypes('boolean'))

In [341]:
#train_values[pd.get_dummies(train_values.select_dtypes('bool')).columns] = pd.get_dummies(train_values.select_dtypes('bool'))

In [385]:
train_values.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [389]:
features = ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status', 'range_age']
columns.append(features)

for feature in features:
  del train_values[feature]

In [390]:
scaler = MinMaxScaler()
scaler.fit(train_values)
x_train = scaler.transform(train_values)

Para el método SelectKBest las métricas posibles para clasificación están en: https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

Elegí chi2 porque es la más usada, no sé qué tan bien está porque usamos otra métrica, pero no me deja ponerla.

Creo que estaría bueno hacerlo para las 3 métricas que figuran ahí, y quedarnos con la unión de las 3 métricas (mutual_info_classif está piola porque mide la dependencia entre los features) -> Hasta hay más formas de calcularlos.

La conclusión es que habría o que ver qué forma de calcularlos nos conviene más, o hacer varias e ir quedándonos con intersección o unión de todo eso que calculamos.

In [391]:
print(x_train.shape)
print(train_labels.shape)

(260601, 87)
(260601, 1)


In [392]:
selector = SelectKBest(chi2, 40)
selector.fit(x_train, train_labels)

SelectKBest(k=40, score_func=<function chi2 at 0x7f8d0b6a18c0>)

In [393]:
x_train_fs = selector.transform(x_train)

In [394]:
k_features = np.array(selector.get_support()).reshape((1,-1))
k_features

array([[ True, False, False, False, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False,
        False,  True, False,  True,  True, False, False, False, False,
        False, False, False, False,  True,  True, False, False, False,
         True,  True,  True, False,  True,  True,  True, False, False,
        False, False,  True,  True,  True,  True,  True,  True,  True,
         True, False,  True, False, False,  True,  True,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True,  True, False, False, False,
         True, False,  True,  True,  True, False]])

In [395]:
selected_features = []
features = train_values.columns.tolist()
for i, feature in enumerate(features):
  if k_features[0][i]:
    selected_features.append(feature)

selected_features

['geo_level_1_id',
 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_secondary_use',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'families_and_secondary_use',
 'inhabited_building',
 'count_families_geo_level_1_id',
 'count_families_geo_level_2_id',
 'count_families_geo_level_3_id',
 'count_buildings_geo_level_2_id',
 'count_buildings_geo_level_3_id',
 'has_superstructure_engineered',
 'foundation_type_i',
 'foundation_type_r',
 'foundation_type_u',
 'foundation_type_w',
 'roof_type_n',
 'roof_type_q',
 'roof_type_x',
 'ground_floor_type_f',
 'ground_floor_type_v',
 'other_floor_type_j',
 'other_floor_type_q',
 'other_floor_type_s',
 'plan_configurati

De acá podemos obtener que los mejores features son:

In [398]:
selected_features = ['geo_level_1_id', 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_secondary_use',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'families_and_secondary_use',
 'inhabited_building',
 'count_families_geo_level_1_id',
 'count_families_geo_level_2_id',
 'count_families_geo_level_3_id',
 'count_buildings_geo_level_2_id',
 'count_buildings_geo_level_3_id',
 'has_superstructure_engineered',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'plan_configuration',
 'legal_ownership_status',
 'range_age']

In [ ]:
train_values_selected_features = train_values[selected_features]
train_values_selected_features.head()

,geo_level_1_id,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_v,other_floor_type_j,other_floor_type_q,other_floor_type_s,position_t,plan_configuration_u,legal_ownership_status_a
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
28830,8,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
94947,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
590882,22,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201944,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_values_selected_features = test_values[selected_features]
test_values_selected_features.head()

,geo_level_1_id,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_v,other_floor_type_j,other_floor_type_q,other_floor_type_s,position_t,plan_configuration_u,legal_ownership_status_a
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
300051,17,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
99355,6,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
890251,22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
745817,26,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
421793,17,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


###Método 2:
NO CORRER

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

Jugar un poco con los parámetros

In [ ]:
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='f1_micro',
              min_features_to_select=15)

In [ ]:
rfecv.fit(x_train, train_labels)

In [ ]:
rfecv.n_features_

# Pasaje a CSV de los nuevos Features

In [425]:
train_values_selected_features = train_values[selected_features]
test_values_selected_features = test_values[selected_features]

In [426]:
train_values_selected_features.to_csv('train_values_selected_features.csv')

In [427]:
test_values_selected_features.to_csv('test_values_selected_features.csv')